In [14]:
import torch
import torch.nn as nn
import numpy as np
from sklearn.model_selection import train_test_split
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

data = np.load('all_players_tensor.npy')

X = data[:, :, :]
X = np.delete(X, -2, axis=2)
y = data[:, :, -2]  # Labels: second last column indicating injury (1) or not (0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=20)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])  
        return out

input_size = X.shape[2]  # Num of features
hidden_size = 128
num_layers = 4
output_size = 1

model = RNN(input_size, hidden_size, num_layers, output_size)


pos_weight = torch.tensor([2.0])  
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

optimizer = optim.Adam(model.parameters(), lr=0.001)

train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# Training the model
num_epochs = 10

for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):
        # Forward pass
        outputs = model(inputs)
        labels = labels[:, -1]  # Selecting only the last label in each sequence
        loss = criterion(outputs.squeeze(), labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

# Evaluation of each match sequentially
with torch.no_grad():
    all_player_predictions = []
    correct_predictions = 0
    total_predictions = 0
    
    for i in range(len(X_test)): 
        player_predictions = []
        for j in range(X_test.size(1)):  

            input_match = X_test[i:i+1, j:j+1, :]
            
            output = model(input_match)
            predicted = torch.round(torch.sigmoid(output)).item()
            
            player_predictions.append(predicted)
            
            actual = int(y_test[i, j+1].item()) if j < X_test.size(1) - 1 else None
            if actual is not None:
                correct_predictions += (predicted == actual)
                total_predictions += 1
        
        all_player_predictions.append(player_predictions)
    
    accuracy = correct_predictions / total_predictions
    print(f'Accuracy on test set: {accuracy:.2f}')
print(all_player_predictions)


Epoch [1/10], Step [10/16], Loss: 0.4220
Epoch [2/10], Step [10/16], Loss: 0.1941
Epoch [3/10], Step [10/16], Loss: 0.3735
Epoch [4/10], Step [10/16], Loss: 0.4290
Epoch [5/10], Step [10/16], Loss: 0.4260
Epoch [6/10], Step [10/16], Loss: 0.1104
Epoch [7/10], Step [10/16], Loss: 0.6499
Epoch [8/10], Step [10/16], Loss: 0.1106
Epoch [9/10], Step [10/16], Loss: 0.4168
Epoch [10/10], Step [10/16], Loss: 0.0238
Accuracy on test set: 0.83
[[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0